In [4]:
import warnings
import re
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
warnings.filterwarnings('always', category=DeprecationWarning, module=r'^{0}\.'.format(re.escape(__name__)))
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [8]:
roles = ['School_Administrators', 'Clinical_Staff', 'School_Counselor', 'School Psychologist', 'Teachers']
context = '''Using the information contained in the context,give a comprehensive answer to the question.
                         Respond only to the question asked, response should be concise and relevant to the question. 
                         Provide the number of the source document when relevant. 
                         If the answer cannot be deduced from the context, do not give an answer. 

                        We will provide you the context from the course content of Multi-Tiered System of Supports (MTSS) 
                        a comprehensive, evidence-based framework for
                        supporting successful student outcomes and eliminating barriers to learning. It is designed to
                        identify and address the needs of all students, including those who may be struggling
                        academically and behaviorally, by providing interventions and supports at varying levels of
                        intensity.'''
system_templates = {
            "School_Administrators": "You have been asked a question by School Administrators who job role is to ensures there is an MTSS team to design the school-wide implementation process, progress monitoring protocols, and data collection procedures. Below are the context:\nThis is context1:\n{context1}\n\nThis is context2:\n{context2}\n\nThis is context3:\n{context3}\n\nThis is context4:\n{context4}",
            "Clinical_Staff": "You are a product manager for a social media platform that specializes in coming up metrics and corresponding SQL queries based on data table DDLs. Below are the DDLs:\nThis is DDL1:\n{ddl1}\n\nThis is DDL2:\n{ddl2}\n\nThis is DDL3:\n{ddl3}",
            "School_Counselor": "You have been asked a question by School Counselors who job role is to As a primary resource for administrators, teachers, and parents regarding mental health awareness, school counselors implement school counseling programs addressing the needs of all students. School counselors deliver instruction, appraisal, and advisement to students in all tiers and collaborate with other specialized instructional and intervention personnel, educators, and families to ensure appropriate academic and behavioral supports for students within the school’s MTSS framework. Below are the context:\nThis is context1:\n{context1}\n\nThis is context2:\n{context2}\n\nThis is context3:\n{context3}\n\nThis is context4:\n{context4}",
            'School Psychologist':"",
            'Teachers':""
            
        }

In [9]:
f = open('openai_key.txt')

In [10]:
os.environ['OPENAI_API_KEY'] = f.read()
chat = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'], model='gpt-4-turbo',temperature=0)

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

loader = TextLoader('data/Final_txt_document_course.txt')
data = loader.load()

MARKDOWN_SEPARATORS = [
"#####",
"*****",
"@@@@@",
"\n\n\n\n\n\n",
"\n\n\n\n\n",  
"\n\n\n\n",  
"\n\n\n",  
"\n\n",
    "\n",
    " ",
    "",
]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=20000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
            chunk_overlap=100,  # The number of characters to overlap between chunks
            strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
            separators=MARKDOWN_SEPARATORS,
        )

docs_processed = text_splitter.split_documents(data)
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
            docs_processed, embeddings, distance_strategy=DistanceStrategy.COSINE
        )


In [16]:
user_query = "Briefly describe MTSS and the process of integrating behavioral health support into an existing academic framework."
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


MTSS, or Multi-Tiered System of Supports, is a comprehensive, evidence-based framework designed to meet the diverse needs of all students through layered levels of support. It aims to enhance academic, behavioral, and social-emotional outcomes by providing interventions at varying intensities tailored to student needs. MTSS is structured around three tiers: Tier 1 (universal supports for all students), Tier 2 (targeted supports for students who need more assistance), and Tier 3 (intensive supports for individual students with significant needs).

Integrating behavioral health support into an existing academic MTSS involves a systematic approach that recognizes the importance of addressing both academic and mental health needs to support overall student well-being. This integration process typically includes:

1. **Assessing the Current State**: Reviewing the existing academic MTSS framework and supports for behavioral and mental health to identify gaps and strengths.

2. **Establishing

In [17]:
user_query = "Discuss the importance of integrating behavioral health into a tiered academic support framework."
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Integrating behavioral health into a tiered academic support framework, such as the Multi-Tiered System of Supports (MTSS), is crucial for several reasons. This integration not only enhances the overall effectiveness of educational interventions but also addresses the comprehensive needs of students, promoting both academic success and emotional well-being. Here are some key points highlighting the importance of this integration:

1. **Holistic Student Support**: Behavioral health is a critical component of overall student well-being. By integrating behavioral health into a tiered support framework, schools can provide a more holistic approach that addresses both academic and non-academic barriers to learning. This approach recognizes that students' emotional and mental health needs significantly impact their ability to learn and succeed academically.

2. **Early Identification and Intervention**: A tiered framework allows for the early identification of behavioral and emotional issues

In [18]:
user_query = "Discuss the barriers to implement an integrated MTSS framework."
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Implementing an integrated Multi-Tiered System of Supports (MTSS) framework involves a comprehensive approach that combines academic, behavioral, and social-emotional supports into a cohesive system. While this integration can significantly enhance student outcomes, several barriers can impede successful implementation. Here are some of the common challenges:

1. **Lack of Understanding and Buy-in:**
   - **Barrier:** Stakeholders, including staff and families, may not fully understand the purpose and benefits of an integrated MTSS framework. Without a clear understanding, there may be resistance to change and a lack of commitment.
   - **Solution:** Provide comprehensive training and ongoing professional development to all stakeholders. Regular communication about the goals, processes, and successes of MTSS can help build buy-in and understanding.

2. **Inadequate Resources:**
   - **Barrier:** Implementing MTSS requires sufficient resources, including funding, personnel, and material

In [19]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Implementing an effective integrated Multi-Tiered System of Supports (MTSS) involves several key concepts that ensure the framework is comprehensive and addresses both academic and behavioral/mental health needs of students. Here are the essential concepts:

1. **Leadership and Team Structure**: Establish a dedicated leadership team that includes administrators, teachers, counselors, and other stakeholders. This team is responsible for overseeing the MTSS implementation, ensuring fidelity, and facilitating communication among all participants.

2. **Shared Vision and Goals**: Develop a clear and shared vision for MTSS implementation that includes specific, measurable goals. This shared vision helps align the efforts of all stakeholders and guides decision-making processes.

3. **Assessment of Readiness and Capacity**: Evaluate the school or district’s readiness and capacity to implement MTSS. This includes assessing available resources, staff competencies, and existing infrastructure.


In [20]:
user_query = "We are just starting with MTSS. What are the core features of the framework. Where should we begin? "
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

Implementing a Multi-Tiered System of Supports (MTSS) is a strategic approach to enhancing educational outcomes for all students. It integrates academic, behavioral, and social-emotional support in a comprehensive framework. Here are the core features of the MTSS framework and some guidance on where to begin:

### Core Features of the MTSS Framework:

1. **Tiered Delivery System:**
   - **Tier 1:** Universal supports for all students to promote academic success and well-being through high-quality instruction and proactive strategies.
   - **Tier 2:** Targeted interventions for students who need additional support beyond the universal programs.
   - **Tier 3:** Intensive, individualized interventions for students who require more specialized assistance.

2. **Data-Informed Decision Making:**
   - Regular collection and analysis of data on student performance and behavior to guide decisions at all levels of the MTSS framework.

3. **Evidence-Based Practices:**
   - Implementation of inst

In [21]:
user_query = "I need family voice that represents our diverse school community.  I have lots of input from a vocal but small minority of parents expressing concerns about integrating positive health and wellness (SEL) in our Tier 1 curriculum. I believe most families are in favor of this, but they are not as vocal. What can I do to get more diverse voices especially around positive health and wellness (SEL)?  "
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

To gather a more diverse range of family voices, especially concerning the integration of positive health and wellness (Social Emotional Learning or SEL) in your Tier 1 curriculum, you can employ several strategies to ensure broad and inclusive family engagement. Here are some practical steps you can take:

1. **Conduct Surveys and Questionnaires:**
   - Develop and distribute surveys or questionnaires that ask specific questions about families' views on integrating SEL into the curriculum. Ensure these are available in multiple languages and accessible formats to accommodate all families. Online platforms can be used for easy distribution and collection of responses.

2. **Host Focus Groups:**
   - Organize focus group sessions with parents and guardians from diverse backgrounds. These sessions can provide deeper insights into their thoughts and feelings about SEL. Ensure that the groups are facilitated by someone who can create an inclusive atmosphere and encourage open dialogue.

3.

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)

In [ ]:
user_query = "What are the key concepts in implementing an effective integrated MTSS?"
role = 'School_Administrators'

query_vector = embeddings.embed_query(user_query)
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=4)
context1, context2, context3, context4 = retrieved_docs[0].page_content, retrieved_docs[1].page_content, retrieved_docs[2].page_content, retrieved_docs[3].page_content

system_template = system_templates[role]
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
human_template=user_query
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.format_prompt(context1=context1, context2=context2, context3=context3, context4=context4, question=user_query).to_messages()
        
result = chat(request)
print(result.content)